In [1]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import cvxpy to do constrained optimization
import cvxpy as cp

In [2]:
# Read in previously computed percent profiles, 
# which reflect the accounting for reaction efficiencies
# Only load final OH and PPP rows
pct_df = pd.read_csv('csv_results/pct_profiles_corrected.csv', index_col=[0,1]).iloc[-6:,:]
pct_df

p4        p5         p6         p7         p8        p9  \
end sample                                                                  
PPP rep1    0.038938  0.344089   3.037849  46.641690  41.775202  7.786260   
    rep2    0.054160  0.403518   3.438273  46.805748  41.190190  7.727563   
    rep3    0.049701  0.353132   2.981290  46.590446  41.794130  7.825740   
OH  rep1    0.251587  2.979228  36.366299  44.325032  13.870265  1.509781   
    rep2    0.251821  2.873950  35.269190  43.459023  15.720999  1.729855   
    rep3    0.286697  3.234526  39.061322  43.415968  12.692872  0.648448   

                 p10  
end sample            
PPP rep1    0.375971  
    rep2    0.380547  
    rep3    0.405560  
OH  rep1    0.697808  
    rep2    0.695162  
    rep3    0.660167

In [3]:
# Compute means and stds across replicates
reps = ['rep1','rep2','rep3']
for treatment in ['OH','PPP']:
    rows = [(treatment,rep) for rep in reps]
    pct_df.loc[(treatment,'mean'),:] = pct_df.loc[rows,:].mean(axis=0) 
    pct_df.loc[(treatment,'std'),:] = pct_df.loc[rows,:].std(axis=0)
pct_df = pct_df.sort_index()
pct_df

p4        p5         p6         p7         p8        p9  \
end sample                                                                  
OH  mean    0.263368  3.029235  36.898937  43.733341  14.094712  1.296028   
    rep1    0.251587  2.979228  36.366299  44.325032  13.870265  1.509781   
    rep2    0.251821  2.873950  35.269190  43.459023  15.720999  1.729855   
    rep3    0.286697  3.234526  39.061322  43.415968  12.692872  0.648448   
    std     0.020203  0.185416   1.951370   0.512872   1.526490  0.571514   
PPP mean    0.047600  0.366913   3.152471  46.679295  41.586507  7.779855   
    rep1    0.038938  0.344089   3.037849  46.641690  41.775202  7.786260   
    rep2    0.054160  0.403518   3.438273  46.805748  41.190190  7.727563   
    rep3    0.049701  0.353132   2.981290  46.590446  41.794130  7.825740   
    std     0.007826  0.032022   0.249122   0.112469   0.343352  0.049401   

                 p10  
end sample            
OH  mean    0.684379  
    rep1    0.697808  
    rep2    0.695162  
    rep3    0.660167  
    std     0.021010  
PPP mean    0.387359  
    rep1    0.375971  
    rep2    0.380547  
    rep3    0.405560  
    std     0.015927

In [4]:
# Create dataframe to hold shift results
max_shift_left = 4
max_shift_right = 4
shifts = np.array(range(-max_shift_left, max_shift_right+1))
coeff_df = pd.DataFrame(columns=shifts, index=reps, data=0)

# Fill out 
min_p5_pos = 4
max_p5_pos = 10
p5_pos = np.arange(min_p5_pos, max_p5_pos+1)
max_shift_left = 4
max_shift_right = 4
shifts = np.array(range(-max_shift_left, max_shift_right+1))
M = len(pct_df.columns)
N = len(shifts)

summary_df = pd.DataFrame(columns=['-1 shift', '+0 shift', 'other shifts', 'residual'],
                          index=reps)

### Plot PPP and OH 5' positions
for rep in ['rep1','rep2','rep3']:

    # PPP histogram
    ppp_hist = pct_df.loc[('PPP',rep),:] #np.array([.04, .32, 3.06, 46.32, 41.91, 7.97, .37])
    ppp_hist = ppp_hist/ppp_hist.sum()

    # OH histogram
    oh_hist =  pct_df.loc[('OH',rep),:] #np.array([.30, 3.57, 43.12, 45.09, 7.29, 0, .64])
    oh_hist = oh_hist/oh_hist.sum()

    ### Create A matrix. Columns correspond to shifted pct profiles
    A = np.zeros([M,N])
    for i, shift in enumerate(shifts):
        if shift < 0:
            A[:shift,i] = ppp_hist[-shift:]
        elif shift > 0:
            A[shift:,i] = ppp_hist[:-shift]
        else:
            A[:,i] = ppp_hist

    # Normalize by rows
    A = A / A.sum(axis=0)

    # Create b matrix
    b = np.mat(oh_hist).T

    # Define and solve the constrained convex optimization problem
    x = cp.Variable([N,1]) # Define vector of mixture probabilities
    cost = cp.sum_squares(A @ x - b) # Define least squares cost
    constraints = [sum(x)==1, x>=0] # Constrain mixture probabilities to sum to 1 and be positive
    prob = cp.Problem(cp.Minimize(cost), constraints) # Perform minimizaiton
    prob.solve()
    mixture_coeffs = np.ravel(x.value)
    
    # Enforce zero min constraint
    mixture_coeffs[mixture_coeffs < 0] = 0
    
    # Save to dataframe
    coeff_df.loc[rep,:] = 100*mixture_coeffs
    
    # Compute the approximation
    fit_hist = A @ mixture_coeffs
    pct_df.loc[('mixture',rep),:] = 100*fit_hist
    
    ix = [i for i in coeff_df.columns if i not in [-1,0]]
    summary_df.loc[rep,'-1 shift'] = coeff_df.loc[rep,-1]
    summary_df.loc[rep,'+0 shift'] = coeff_df.loc[rep,0]
    summary_df.loc[rep, 'other shifts'] = coeff_df.loc[rep,ix].sum()
    summary_df.loc[rep, 'residual'] = 100*np.sum(np.abs(fit_hist - oh_hist))/2
    
pct_df.loc[('mixture','mean'),:] = pct_df.loc[('mixture', reps),:].mean(axis=0)
pct_df.loc[('mixture','std'),:] = pct_df.loc[('mixture', reps),:].std(axis=0)
pct_df = pct_df.sort_index()
    
coeff_df.loc['mean',:] = coeff_df.loc[reps,:].mean(axis=0)
coeff_df.loc['std',:] = coeff_df.loc[reps,:].std(axis=0)

summary_df.loc['mean',:] = summary_df.loc[reps,:].mean(axis=0)
summary_df.loc['std',:] = summary_df.loc[reps,:].std(axis=0)

In [5]:
pct_df.to_csv('csv_results/mixture_profiles.csv')
pct_df

p4        p5         p6         p7         p8        p9  \
end     sample                                                                  
OH      mean    0.263368  3.029235  36.898937  43.733341  14.094712  1.296028   
        rep1    0.251587  2.979228  36.366299  44.325032  13.870265  1.509781   
        rep2    0.251821  2.873950  35.269190  43.459023  15.720999  1.729855   
        rep3    0.286697  3.234526  39.061322  43.415968  12.692872  0.648448   
        std     0.020203  0.185416   1.951370   0.512872   1.526490  0.571514   
PPP     mean    0.047600  0.366913   3.152471  46.679295  41.586507  7.779855   
        rep1    0.038938  0.344089   3.037849  46.641690  41.775202  7.786260   
        rep2    0.054160  0.403518   3.438273  46.805748  41.190190  7.727563   
        rep3    0.049701  0.353132   2.981290  46.590446  41.794130  7.825740   
        std     0.007826  0.032022   0.249122   0.112469   0.343352  0.049401   
mixture mean    0.318222  2.571846  37.441155  42.513111  14.797994  1.948896   
        rep1    0.277853  2.453139  37.185241  42.677836  15.016152  1.977627   
        rep2    0.313381  2.655502  35.627318  42.484272  16.213136  2.272071   
        rep3    0.363431  2.606897  39.510905  42.377225  13.164695  1.596991   
        std     0.042994  0.105637   1.954400   0.152366   1.535885  0.338456   

                     p10  
end     sample            
OH      mean    0.684379  
        rep1    0.697808  
        rep2    0.695162  
        rep3    0.660167  
        std     0.021010  
PPP     mean    0.387359  
        rep1    0.375971  
        rep2    0.380547  
        rep3    0.405560  
        std     0.015927  
mixture mean    0.408776  
        rep1    0.412153  
        rep2    0.434319  
        rep3    0.379856  
        std     0.027388

In [6]:
coeff_df.to_csv('csv_results/mixture_coefficients.csv')
coeff_df

,-4,-3,-2,-1,0,1,2,3,4
rep1,0.000000e+00,0.000000,0.0,78.305001,21.338729,0.000000e+00,0.0,0.356270,0.000000e+00
rep2,0.000000e+00,0.000000,0.0,74.209520,25.424816,1.141728e-21,0.0,0.365664,3.634946e-23
rep3,5.205406e-22,0.123904,0.0,83.730058,15.807355,0.000000e+00,0.0,0.338682,0.000000e+00
mean,1.735135e-22,0.041301,0.0,78.748193,20.856967,3.805761e-22,0.0,0.353539,1.211649e-23
std,3.005343e-22,0.071536,0.0,4.775717,4.826796,6.591772e-22,0.0,0.013696,2.098637e-23


In [7]:
summary_df.to_csv('csv_results/mixture_summary.csv')
summary_df

,-1 shift,+0 shift,other shifts,residual
rep1,78.305001,21.338729,0.35627,2.458941
rep2,74.20952,25.424816,0.365664,1.454041
rep3,83.730058,15.807355,0.462587,1.946684
mean,78.748193,20.856967,0.39484,1.953222
std,4.775717,4.826796,0.058858,0.502482


**Main text** 

We modeled the OH profile as a mixture of PPP profiles, each shifted between -4 nt and +4 nt. The mixture coefficients were inferred using lest-squares regression under positivity and normalization constraints. The PPP profile shifted -1 nt shift accounted for 78.7% $\pm$ 4.8% of the resulting mixture model, the 0 nt shifted profile for an additional 20.0% $\pm$ 4.8%, and all other shifted profiles for 0.39% $\pm$ 0.06%. The residual deviation of 2.0% $\pm$ 0.5% between our mixture model and the measured OH profile is consistent with nanoRNA priming having minor differences in sequence requirements relative to NTP priming. We therefore conclude that the bulk OH profile can be almost full explained by the priming of 2 nt nanoRNAs.

**SI Text**

To carry out the profile mixture modeling, we defined a 7x9 matrx $A$, whose entires $A_{ij}$ represent the fraction of reads with 5' positions at 4, 5, ..., 10 (corresponding to $i$ = 1, 2, ..., 7) within the PPP profile shifted by $-4, -3, ..., +4$ nt (corresponding to $j$ = 1, 2, ..., 9). Note that these shifted profiles were normalized so that $\sum_i A_{ij} = 1$ for every column $j$. We further defined a 7x1 vector $b$ representing the OH profile, normalized so that $\sum_i b_i = 1$. We then inferred a 9x1 vector of mixture coefficients $x$ by solving constrained least squares problem 

$$ x = \mathrm{argmin}(||A x - b||^2) $$

under the constraints that $x_i \geq 0$ for all $i$ and $\sum_i x_i = 1$. The resulting mixture profile $b^* = A x$ is shown in Fig SX (top panel), alongside the OH profile $b$. The corresponding mixture coefficients are shown in Fig SX(bottom panel). The residual deviation was computed as 

$$ \frac{1}{2} \sum_i |b^*_i - b_i|.$$


**Figure Caption**

Mixture model for OH profile. Left panel shows the observed histograms for PPP and OH ends, determined after inferring efficiencies. Middle panel shows the inferred mixture coefficients. Right panel shows the inferred mixture model profile next to the OH profile. 